In [ ]:
%cd /app

import numpy as np
import pandas as pd

# Building and Manipulating TimeSeries objects

In [ ]:
from darts import TimeSeries
from darts.utils.utils import generate_index

# generate a sine and cosine wave
x = np.linspace(0,2 * np.pi, 100)
sine_vals = np.sin(x)
cosine_values = np.cos(x)

#generate a DatetimeIndex with daily frequency
dates = generate_index(
    start = "2020-01-01",
    length = len(x), 
    freq="D"
)

df = pd.DataFrame({
    "sine": sine_vals,
    "cosine": cosine_values,
    "time": dates
})

series = TimeSeries.from_dataframe(df, time_col = "time")

In [ ]:
series.plot()

In [ ]:
from darts.datasets import AirPassengersDataset

series = AirPassengersDataset().load()
series.plot()

In [ ]:
series[-5:].to_dataframe(backend="pandas", time_as_index=True)

In [ ]:
series1, series2 = series.split_after(0.75)
series1.plot(label="Train")
series2.plot(label="Test")

In [ ]:
series1, series2 = series[:-36], series[-36:]
series1.plot(label="Train")
series2.plot(label="Test")

In [ ]:
series_noise = TimeSeries.from_times_and_values(
    series.time_index, np.random.randn(len(series))
)
combined_series = series / 2 + 20 * series_noise - 1
combined_series.plot(label="Combined Series")

In [ ]:
stacked_series = (series / 50).stack(series_noise)
stacked_series.plot()

In [ ]:
series.map(np.log).plot()

In [ ]:
series.map(lambda ts, x: x / ts.days_in_month).plot()

In [ ]:
(series / 20).add_datetime_attribute("month").plot()

In [ ]:
(series / 200).add_holidays("US").plot()

In [ ]:
series.diff().plot()

In [ ]:
## fill missing values using a utils function
from darts.utils.missing_values import fill_missing_values

values = np.arange(50, step=0.5)
values[10:30] = np.nan
values[60:95] = np.nan

series_ = TimeSeries.from_values(values)
(series_ - 10).plot(label = "with missing values (shifted below)")
fill_missing_values(series_).plot(label = "without missing values")

# Creating a training and validation series

In [ ]:
train, val = series.split_before(pd.Timestamp("19580101"))
train.plot(label="Train")
val.plot(label="Validation")

# Training forecasting models and making predictions

## Playing with toy models

In [ ]:
from darts.models import NaiveSeasonal

naive_model = NaiveSeasonal(K=1)
naive_model_s = NaiveSeasonal(K=12)
naive_model.fit(train)
naive_forecast_1 = naive_model.predict(36)

naive_model_s.fit(train)
naive_forecast_12 = naive_model_s.predict(36)

series.plot(label="actual")
naive_forecast_1.plot(label="naive forecast (K=1)")
naive_forecast_12.plot(label="naive forecast (K=12)")

## Inspect Seasonality

In [ ]:
from darts.utils.statistics import check_seasonality, plot_acf

plot_acf(train, m=12, alpha=0.05, max_lag=24)

In [ ]:
for m in range(2, 25):
    is_seasonal, period = check_seasonality(train, m=m, alpha=0.05)
    if is_seasonal:
        print(f"There is seasonality of {period} months")

## A less naive model

In [ ]:
seasonal_model = NaiveSeasonal(K=12)
seasonal_model.fit(train)
seasonal_forecast = seasonal_model.predict(36)

series.plot(label="actual")
seasonal_forecast.plot(label="naive seasonal forecast K=12")

In [ ]:
from darts.models import NaiveDrift

drift_model = NaiveDrift()
drift_model.fit(train)
drift_forecast = drift_model.predict(36)

combined_forecast = drift_forecast + seasonal_forecast - train.last_value()

series.plot()
combined_forecast.plot(label="combined")
drift_forecast.plot(label="drift")

## Computing error metrics

In [ ]:
from darts.metrics import mape

print(
    f"Mean absolute percentage error for the combined naive drift + seasonal: {mape(series, combined_forecast):.2f}%."
)

## Quickly try out several models

In [ ]:
from darts.models import AutoARIMA, ExponentialSmoothing, Theta

def eval_model(model):
    model.fit(train)
    forecast = model.predict(len(val))
    print(f"Model {model} obtains MAPE: {mape(val, forecast):.2f}%")

eval_model(ExponentialSmoothing())
eval_model(AutoARIMA())
eval_model(Theta())

## Searching for hyper-parameters with the Theta method



In [ ]:
thetas = 2 - np.linspace(-10, 10, 50)

best_mape = float("inf")
best_theta = 0

for theta in thetas:
    model = Theta(theta)
    model.fit(train)
    pred_theta = model.predict(len(val))
    res = mape(val, pred_theta)
    if res < best_mape:
        best_mape = res
        best_theta = theta

In [ ]:
best_theta_model = Theta(best_theta)
best_theta_model.fit(train)
pred_best_theta = best_theta_model.predict(len(val))

print(f"Lowest MAPE is: {mape(val, pred_best_theta):.2f}% with theta={best_theta:.2f}")

In [ ]:
train.plot(label="train")
val.plot(label="true")
pred_best_theta.plot(label="prediction")

# Backtesting: simulate historical forecasting

In [ ]:
hfc_params = {
    "series": series,
    "start": pd.Timestamp("1956-01-01"),
    "forecast_horizon": 3,
    "verbose": True
}
# last points only, only use the 3rd month of the fcast to make series
historical_fcast_theta = best_theta_model.historical_forecasts(last_points_only=True, **hfc_params)

series.plot(label="data")
historical_fcast_theta.plot(label="backtest 3-months ahead forecast (Theta)")
print(f"MAPE: {mape(series, historical_fcast_theta):.2f}%")

In [ ]:
historical_fcast_theta

In [ ]:
historical_fcast_theta_all = best_theta_model.historical_forecasts(
    last_points_only=False, stride=3, **hfc_params
)

series.plot(label="data")
for idx, hfc in enumerate(historical_fcast_theta_all):
    hfc.plot(label=f"forecast {idx}")


from darts import concatenate

historical_fcast_theta_all_2 = concatenate(historical_fcast_theta_all, axis=0)
print(f"MAPE: {mape(series, historical_fcast_theta_all_2):.2f}%")

In [ ]:
historical_fcast_theta_all

In [ ]:
best_theta_model = Theta(best_theta)

raw_errors = best_theta_model.backtest(
    metric=mape, reduction=None, last_points_only=False, stride=1, **hfc_params
)

from darts.utils.statistics import plot_hist

plot_hist(
    raw_errors,
    bins=np.arange(0, max(raw_errors),1),
    title="Individual backtest error scores (histogram)"
)

In [ ]:
average_error = best_theta_model.backtest(
    metric=mape,
    reduction=np.mean, # np.median
    **hfc_params
)

print(f"Average backtest error: {average_error:.2f}%")

In [ ]:
hfc_precomputed = best_theta_model.historical_forecasts(
    last_points_only=False, stride=1, **hfc_params
)
new_error = best_theta_model.backtest(
    historical_forecasts=hfc_precomputed,
    last_points_only=False,
    stride=1,
    **hfc_params
)

print(f"Average error (MAPE) over all historical forecasts: {new_error:.2f}%")

In [ ]:
print(len(hfc_precomputed))
hfc_precomputed[1].to_dataframe()

In [ ]:
from darts.utils.statistics import plot_residuals_analysis

plot_residuals_analysis(best_theta_model.residuals(series))

In [ ]:
from darts.metrics import ae
import matplotlib.pyplot as plt

residuals = best_theta_model.residuals(
    historical_forecasts=hfc_precomputed,
    metric = ae,
    last_points_only=False,
    values_only=True,
    **hfc_params
)
residuals2 = np.concatenate(residuals, axis=1)[:,:,0] # remove the third dimension 

fig, ax = plt.subplots()
for forecast_step in range(len(residuals2)):
    ax.hist(residuals2[forecast_step], label=f"step {forecast_step}", alpha=0.5)
ax.legend()
ax.set_title("Absolute Errors per forecast step")

In [ ]:
np.concatenate(residuals, axis=1)[:,:,0].shape


In [ ]:
model_es = ExponentialSmoothing(seasonal_periods=12)
historical_fcast_es = model_es.historical_forecasts(**hfc_params)

series.plot(label="data")
historical_fcast_es.plot(label="backtest 3-months ahead forecast (exp. smoothing)")
print(f"MAPE: {mape(historical_fcast_es, series):.2f}%")


In [ ]:
plot_residuals_analysis(model_es.residuals(series, verbose=True))

# Machine Learning and global models

## a toy example with two series

In [ ]:
from darts.datasets import AirPassengersDataset, MonthlyMilkDataset

series_air = AirPassengersDataset().load().astype(np.float32)
series_milk = MonthlyMilkDataset().load().astype(np.float32)

train_air, val_air = series_air[:-36], series_air[-36:]
train_milk, val_milk = series_milk[:-36], series_milk[-36:]

train_air.plot()
train_milk.plot()
val_air.plot()
val_milk.plot()

In [ ]:
from darts.dataprocessing.transformers import Scaler

scaler = Scaler() # scale between 0 and 1

train_air_scaled, train_milk_scaled = scaler.fit_transform([train_air, train_milk])

train_air_scaled.plot()
train_milk_scaled.plot()

## Using deep learning: n-beats

In [ ]:
from darts.models import NBEATSModel

model = NBEATSModel(
    input_chunk_length=24,
    output_chunk_length=12,
    random_state=42
)

model.fit([train_air_scaled, train_milk_scaled], epochs=50, verbose=True)

In [ ]:
pred_air, pred_milk = model.predict(
    series = [train_air_scaled, train_milk_scaled],
    n=36
)

# scale back
pred_air, pred_milk = scaler.inverse_transform([pred_air, pred_milk])

plt.figure(figsize=(10, 6))
series_air.plot(label="Air Passengers (actual)")
pred_air.plot(label="Air Passengers (predicted)")
series_milk.plot(label="Monthly Milk (actual)")
pred_milk.plot(label="Monthly Milk (predicted)")

# Covariates: using external data 

In [ ]:
from darts import concatenate
from darts.utils.timeseries_generation import datetime_attribute_timeseries

air_covs = concatenate(
    [
        datetime_attribute_timeseries(series_air, "month", dtype=np.float32),
        datetime_attribute_timeseries(series_air, "year", dtype=np.float32),
    ],
    axis="component"
)
milk_covs = concatenate(
    [
        datetime_attribute_timeseries(series_milk, "month", dtype=np.float32),
        datetime_attribute_timeseries(series_milk, "year", dtype=np.float32),
    ],
    axis="component"
)

air_covs_scaled, milk_covs_scaled = Scaler().fit_transform([air_covs, milk_covs])
air_covs_scaled.plot()
milk_covs_scaled.plot()
plt.title(
    "one multivariate time sereis of 2 dimensions, containing covariates for the air series"
)

In [ ]:
model = NBEATSModel(
    input_chunk_length=24,
    output_chunk_length=12,
    random_state=42
)

model.fit(
    [train_air_scaled, train_milk_scaled],
    past_covariates=[air_covs_scaled, milk_covs_scaled],
    epochs=50,
    verbose=True
)

In [ ]:
preds = model.predict(
    series = [train_air_scaled, train_milk_scaled],
    past_covariates = [air_covs_scaled, milk_covs_scaled],
    n=36
)

# scale back
pred_air, pred_milk = scaler.inverse_transform(preds)

plt.figure(figsize=(10,6))
series_air.plot(label="Air Passengers (actual)")
pred_air.plot(label="Air Passengers (predicted)")
series_milk.plot(label="Monthly Milk (actual)")
pred_milk.plot(label="Monthly Milk (predicted)")

In [ ]:
def extract_year(idx):
    return (idx.year - 1950) / 50

encoders = {
    "cyclic": {
        "future": ["month"] # montyh is cyclic, so we encode it as a sine and cosine wave
    },
    "datetime_attribute": {
        "future": ["hour", "dayofweek"]
    },
    "position": { # absolute position encoding, used as past and future
        "past": ["absolute"],
        "future": ["relative"]
    },
    "custom": {
        "past": [extract_year]
    },
    "transformer": Scaler()
}





In [ ]:
# actual example
encoders = {
    "datetime_attribute": {
        "past": ["month", "year"]
    },
    "transformer": Scaler()
}

In [ ]:
model = NBEATSModel(
    input_chunk_length=24,
    output_chunk_length=12,
    random_state=42,
    add_encoders=encoders
)

model.fit([train_air_scaled, train_milk_scaled], epochs=50, verbose=True)

In [ ]:
preds = model.predict(
    series = [train_air_scaled, train_milk_scaled],
    n=36,
    show_warnings=True
)

pred_air, pred_milk = scaler.inverse_transform(preds)
plt.figure(figsize=(10, 6))
series_air.plot(label="Air Passengers (actual)")
pred_air.plot(label="Air Passengers (predicted)")
series_milk.plot(label="Monthly Milk (actual)")
pred_milk.plot(label="Monthly Milk (predicted)")

## Regression forecasting models



In [ ]:
from sklearn.linear_model import BayesianRidge
from darts.models import RegressionModel

model = RegressionModel(
    lags=72, 
    lags_future_covariates=[-6,0],
    model=BayesianRidge())

model.fit(
    [train_air_scaled, train_milk_scaled],
    future_covariates=[air_covs_scaled, milk_covs_scaled]
)

In [ ]:
preds = model.predict(
    series=[train_air_scaled, train_milk_scaled],
    future_covariates=[air_covs_scaled, milk_covs_scaled],
    n=36
)

pred_air, pred_milk = scaler.inverse_transform(preds)

plt.figure(figsize=(10, 6))
series_air.plot(label="actual (air)")
series_milk.plot(label="actual (milk)")
pred_air.plot(label="forecast (air)")
pred_milk.plot(label="forecast (milk)");

In [ ]:
mape([series_air, series_milk], [pred_air, pred_milk])

In [ ]:
mape([series_air, series_milk], [pred_air, pred_milk], series_reduction=np.mean)

In [ ]:
bayes_ridge_model = RegressionModel(
    lags=72, lags_future_covariates=[0], model=BayesianRidge()
)

backtest = bayes_ridge_model.historical_forecasts(
    future_covariates = [air_covs_scaled, milk_covs_scaled],
    **hfc_params
)

In [ ]:
print(f"MAPE = {mape(series_air, backtest):.2f}%")
series_air.plot()
backtest.plot();

In [ ]:

# to avoid data leakage, make sure the scaler is retrained each iteration
backtest_auto_scaling = bayes_ridge_model.historical_forecasts(
    future_covariates = air_covs,
    data_transformers = {"series": Scaler(), "future_covariates": Scaler()},
    retrain=True, ### scalers need to be fitted each iteration,
    **hfc_params
)

print(f"MAPE = {mape(series_air, backtest_auto_scaling):.2f}%")
series_air.plot()
backtest_auto_scaling.plot()

## Forecast start shifting

In [ ]:
from darts.models import LinearRegressionModel


model_shifted = LinearRegressionModel(
    lags=12,
    lags_future_covariates=(0,12),
    output_chunk_length=12,
    output_chunk_shift=12
)

model_shifted.fit(series_air[:-24], future_covariates=air_covs)
preds = model_shifted.predict(n=12)

series_air[:-24].plot(label="train")
series_air[-24:].plot(label="validation")
preds.plot(label="shifted prediction")

## probabalistic forecasts

In [ ]:
model_es = ExponentialSmoothing()
model_es.fit(train)
probababilistic_forecast = model_es.predict(len(val), num_samples=500)

series.plot(label="actual")
probababilistic_forecast.plot(label="probabilistic forecast (exp. smoothing)")
plt.legend()
plt.show()